In [1]:
import pandas as pd
from spacy.tokens import DocBin
from tqdm import tqdm
import spacy
from spacy.util import filter_spans
import json

In [2]:
ner_sample = pd.read_csv('ChemNER.csv')

In [3]:
ner_sample.head(3)

,Unnamed: 0,Name,Chemicals,Safety,Sent1,class
0,8014,Micropenam-S Injection,Meropenem (1000mg)+ Sulbactam (500mg),Caution is advised when consuming alcohol with...,Micropenam-S Injection contains Meropenem (100...,ANTI INFECTIVE
1,73385,Uroclan 1.53gm Syrup,Disodium Hydrogen Citrate (1.53gm),It is not known whether it is safe to consume ...,Uroclan 1.53gm Syrup contains Disodium Hydroge...,UROLOGY
2,46431,Rebaxide Tablet,Chlordiazepoxide (2.5mg)+ Clidinium (5mg),It is unsafe to consume alcohol with Rebaxide ...,Rebaxide Tablet contains Chlordiazepoxide (2.5...,GASTRO INTESTINA


In [7]:
sample = ner_sample['Sent1'].astype(str).to_list()

In [8]:
nlp_ner = spacy.load("output/model-best")
docs = nlp_ner.pipe(sample)
for doc in docs:
    print(f"\nSentence: {doc.text}")
    for ent in doc.ents:
        print(f"  - {ent.text} ({ent.label_})")


Sentence: Micropenam-S Injection contains Meropenem (1000mg)+ Sulbactam (500mg) and Caution is advised when consuming alcohol with Micropenam-S Injection
  - Meropenem (Chemical)
  - 1000mg)+ Sulbactam (Chemical)
  - alcohol (Chemical)

Sentence: Uroclan 1.53gm Syrup contains Disodium Hydrogen Citrate (1.53gm) and It is not known whether it is safe to consume alcohol with Uroclan 1
  - Hydrogen Citrate (Chemical)
  - alcohol (Chemical)

Sentence: Rebaxide Tablet contains Chlordiazepoxide (2.5mg)+ Clidinium (5mg) and It is unsafe to consume alcohol with Rebaxide Tablet
  - Rebaxide (Chemical)
  - Chlordiazepoxide (Chemical)
  - 2.5mg)+ Clidinium (Chemical)
  - alcohol (Chemical)
  - Rebaxide (Chemical)

Sentence: Luikyne 1% Cream contains Luliconazole (1% w/w) and No interaction found/established,imageAltTextAlcohol,imageUrlhttps//onemg
  - Luliconazole (Chemical)
  - imageAltTextAlcohol (Chemical)

Sentence: Onstair-O Tablet contains Ondansetron (4mg)+ Omeprazole (20mg) and Caution is

In [ ]:
docs = list(nlp_ner.pipe(sample))


In [28]:
results = []
for doc in docs:
    results.append({
        "text": doc.text,
        "tokens": [token.text for token in doc],
        "entities": [ent.text for ent in doc.ents]
    })

In [29]:
result = pd.DataFrame(results)

In [40]:
result.tail(3)

,text,tokens,entities
97,Vomo 2mg Oral Drops contains Ondansetron (2mg)...,"[Vomo, 2, mg, Oral, Drops, contains, Ondansetr...",[Ondansetron]
98,Succidox 100mg Tablet DT contains Cefpodoxime ...,"[Succidox, 100, mg, Tablet, DT, contains, Cefp...",[]
99,Voglus gm 2mg Tablet contains Glimepiride (2mg...,"[Voglus, gm, 2, mg, Tablet, contains, Glimepir...","[Glimepiride, Metformin, Voglibose]"


In [31]:
result['entities'] = result['entities'].apply(
    lambda lst: [part.strip() for s in lst for part in s.split("+") if part.strip()])

In [36]:
result['entities'] = result['entities'].apply(
    lambda lst: [x for x in lst if x not in ['alcohol']])

In [42]:
ner_sample['tokens'] = result['tokens']
ner_sample['entities'] = result['entities']

In [49]:
ner_result = ner_sample.drop(ner_sample.columns[0], axis=1)

In [51]:
ner_result.to_csv('ner_result.csv',index=False)